In [4]:
!file archive\(6\).zip


archive(6).zip: Zip archive data, at least v4.5 to extract, compression method=deflate


In [9]:
!pip install patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 2.1 MB/s eta 0:00:00


In [10]:
import patoolib
patoolib.extract_archive('archive(6).zip')

INFO patool: Extracting archive(6).zip ...
INFO:patool:Extracting archive(6).zip ...
INFO patool: running /usr/bin/7z x -aou -o./Unpack_9i1uk4_2 -- archive(6).zip
INFO:patool:running /usr/bin/7z x -aou -o./Unpack_9i1uk4_2 -- archive(6).zip
ERROR patool: extraction error, could not remove temporary extraction directory ./Unpack_9i1uk4_2: [Errno 39] Directory not empty: './Unpack_9i1uk4_2'
ERROR:patool:extraction error, could not remove temporary extraction directory ./Unpack_9i1uk4_2: [Errno 39] Directory not empty: './Unpack_9i1uk4_2'


PatoolError: Command `['/usr/bin/7z', 'x', '-aou', '-o./Unpack_9i1uk4_2', '--', 'archive(6).zip']' returned non-zero exit status 2

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 256)  #Adjust based on input size
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 56 * 56)  #Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [54]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset = datasets.ImageFolder(root="/content/Fundus/1000images/", transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)


In [55]:
print(dataset.classes)

['0.0.Normal', '0.1.Tessellated fundus', '0.2.Large optic cup', '0.3.DR1', '1.0.DR2', '1.1.DR3', '10.0.Possible glaucoma', '10.1.Optic atrophy', '11.Severe hypertensive retinopathy', '12.Disc swelling and elevation', '13.Dragged Disc', '14.Congenital disc abnormality', '15.0.Retinitis pigmentosa', '15.1.Bietti crystalline dystrophy', '16.Peripheral retinal degeneration and break', '17.Myelinated nerve fiber', '18.Vitreous particles', '19.Fundus neoplasm', '2.0.BRVO', '2.1.CRVO', '20.Massive hard exudates', '21.Yellow-white spots-flecks', '22.Cotton-wool spots', '23.Vessel tortuosity', '24.Chorioretinal atrophy-coloboma', '25.Preretinal hemorrhage', '26.Fibrosis', '27.Laser Spots', '28.Silicon oil in eye', '29.0.Blur fundus without PDR', '29.1.Blur fundus with suspected PDR']


In [56]:
import torch.optim as optim

model = CustomCNN(num_classes=len(dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [57]:
num_epochs=10
for epoch in range(num_epochs):

    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.2f}%")


    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch 1/10, Training Loss: 5.6766, Training Accuracy: 11.15%
Validation Loss: 3.1603, Validation Accuracy: 17.36%
Epoch 2/10, Training Loss: 2.8745, Training Accuracy: 24.22%
Validation Loss: 2.8322, Validation Accuracy: 28.47%
Epoch 3/10, Training Loss: 2.4098, Training Accuracy: 35.54%
Validation Loss: 2.6275, Validation Accuracy: 34.72%
Epoch 4/10, Training Loss: 1.8559, Training Accuracy: 44.43%
Validation Loss: 2.6780, Validation Accuracy: 34.72%
Epoch 5/10, Training Loss: 1.4558, Training Accuracy: 58.01%
Validation Loss: 3.2920, Validation Accuracy: 31.94%
Epoch 6/10, Training Loss: 1.1268, Training Accuracy: 66.72%
Validation Loss: 3.2452, Validation Accuracy: 34.03%
Epoch 7/10, Training Loss: 1.0223, Training Accuracy: 71.60%
Validation Loss: 3.6663, Validation Accuracy: 33.33%
Epoch 8/10, Training Loss: 0.8461, Training Accuracy: 74.91%
Validation Loss: 4.0035, Validation Accuracy: 28.47%
Epoch 9/10, Training Loss: 0.6610, Training Accuracy: 80.49%
Validation Loss: 4.0624, Va

In [58]:
torch.save(model.state_dict(), "ocular_cnn.pth")

In [59]:
model = CustomCNN(num_classes=len(dataset.classes))
model.load_state_dict(torch.load("ocular_cnn.pth"))
model.eval()

<ipython-input-59-c3564d577793>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("ocular_cnn.pth"))  # Load the saved model state


CustomCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=200704, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=31, bias=True)
)

In [61]:
from torchvision import transforms
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

image_path = "retino.jpg"
image = Image.open(image_path)
image = transform(image).unsqueeze(0)

#Predict
with torch.no_grad():
    output = model(image)
    _, predicted_class = torch.max(output, 1)
    predicted_label = dataset.classes[predicted_class.item()]
    print(f"Predicted Disease: {predicted_label}")


Predicted Disease: 11.Severe hypertensive retinopathy
